In [10]:
import pandas as pd

In [32]:
aed = pd.read_parquet("data/aed_locations.parquet.gzip")
ambulance = pd.read_parquet("data/ambulance_locations.parquet.gzip")
cad = pd.read_parquet("data/cad9.parquet.gzip")

interventions_bxl = pd.read_parquet("data/interventions_bxl.parquet.gzip")
interventions_bxl2 = pd.read_parquet("data/interventions_bxl2.parquet.gzip")

interventions1 = pd.read_parquet("data/interventions1.parquet")
interventions2 = pd.read_parquet("data/interventions2.parquet")
interventions3 = pd.read_parquet("data/interventions3.parquet")

mug = pd.read_parquet("data/mug_locations.parquet.gzip")
pit = pd.read_parquet("data/pit_locations.parquet.gzip")

In [25]:
all_none_columns = aed.isna().all()

print(all_none_columns)

print(aed.head(10))

id              False
type            False
address         False
number          False
postal_code     False
municipality    False
province        False
location        False
public          False
available       False
hours           False
dtype: bool


In [44]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import json

aed_encoded = pd.DataFrame()
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode categorical variable 'C'
aed_encoded["id"] = aed["id"]
aed_encoded["number"] = aed["number"]
aed_encoded["postal_code"] = aed["postal_code"]

aed_encoded["address"] = label_encoder.fit_transform(aed["address"].astype(str))
address_mapping = {label: idx for idx, label in enumerate(label_encoder.classes_)}
with open("address_mapping.json", "w") as f:
    json.dump(address_mapping, f)

aed_encoded["municipality"] = label_encoder.fit_transform(aed["municipality"].astype(str))
municipality_mapping = {label: idx for idx, label in enumerate(label_encoder.classes_)}
with open("municipality_mapping.json", "w") as f:
    json.dump(municipality_mapping, f)

aed_encoded["province"] = label_encoder.fit_transform(aed["province"].astype(str))
province_mapping = {label: idx for idx, label in enumerate(label_encoder.classes_)}
with open("province_mapping.json", "w") as f:
    json.dump(province_mapping, f)


# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=aed_encoded.shape[1])

# Fit and transform the DataFrame
df_imputed = pd.DataFrame(imputer.fit_transform(aed_encoded), columns=aed_encoded.columns)

print(df_imputed)

            id      number  postal_code  address  municipality  province
0         13.0   24.000000       7060.0   1195.0        1901.0       3.0
1         70.0   76.000000       4630.0   1621.0         123.0       5.0
2         71.0   35.333333       4020.0   5219.0        1298.0       5.0
3         72.0   35.000000       4990.0   8544.0        1274.0       5.0
4         73.0  105.333333       4000.0   2551.0        1298.0       5.0
...        ...         ...          ...      ...           ...       ...
15222  16662.0   19.000000       2800.0   4632.0        1427.0       0.0
15223  16664.0   17.000000       9160.0   4651.0        1308.0       9.0
15224  16665.0  207.000000       9000.0   4985.0         764.0       9.0
15225  16666.0   20.000000       1050.0    155.0         365.0       2.0
15226  16668.0   18.000000       5640.0  10519.0        1362.0       7.0

[15227 rows x 6 columns]
